In [1]:
## Libraries
%matplotlib inline
import math
import numpy as np
import matplotlib.pyplot as plt
import pv_model_class
import cvxpy as cp
import pandas as pd

In [2]:
# Constants and variables

Iscn = 8.21 #8.7 # nominal short circuit current
Vocn = 32.9 #37.7 # nominal open circuit voltage
Imp = 7.61 #8.2 # array current at MPP
Vmp = 26.3 #30.1 # array voltage at MPP
Pmax_e = Vmp * Imp # experimental array max power output
Kv = -0.123 #-0.32/100 * Vocn # voltage temperature coefficient
Ki = 0.0032 #-0.032/100 * Iscn # current temperature coefficient
Ns = 54 #60. # number of series cells
Gn = 1000. # nominal irradiance
G = Gn
Tn = 25. + 273.15 # nominal operating temperature
T = Tn

Egap = 1.8 * 10 ** -19 # Bandgap of silicon (silicio cristalino)

err = 0.0001
inverter = 0.95
array_dim = [1, 1]

In [3]:
### Linear Program Example

m = 15
n = 10
np.random.seed(1)
s0 = np.random.randn(m)
lamb0 = np.maximum(-s0, 0)
s0 = np.maximum(s0, 0)
x0 = np.random.randn(n)
A = np.random.randn(m, n)
b = A @ x0 + s0
c = -A.T @ lamb0

# Define and solve the CVXPY problem.
x = cp.Variable(n)
prob = cp.Problem(cp.Minimize(c.T@x),
                 [A @ x <= b])
prob.solve()

# Print result.
print("\nThe optimal value is", prob.value)
print("A solution x is")
print(x.value)
print("A dual solution is")
print(prob.constraints[0].dual_value)


## use '@' when doing multiplication between vectors and matrices -- turns them into scalars
## use '*' when doing multiplication between scalars and vectors or matrices


The optimal value is -15.220912605552861
A solution x is
[-1.10133381 -0.16360111 -0.89734939  0.03216603  0.6069123  -1.12687348
  1.12967856  0.88176638  0.49075229  0.8984822 ]
A dual solution is
[6.98805172e-10 6.11756416e-01 5.28171747e-01 1.07296862e+00
 3.93759300e-09 2.30153870e+00 4.25704434e-10 7.61206896e-01
 8.36906030e-09 2.49370377e-01 1.30187120e-09 2.06014070e+00
 3.22417207e-01 3.84054343e-01 1.59493839e-09]


## Helpful links:
1. https://www.cvxpy.org/tutorial/functions/index.html
2. https://www.cvxpy.org/examples/basic/linear_program.html


# charging constraints
p_cha >= 0
p_cha <= p_lim

# dischargin constraints
p_dis >= 0
p_dis <= p_lim

# SOC constraints
J >= 0
J <= J_max


In [25]:
## getting LMP data, etc

df = pd.read_csv("201801_dalmp_TH_NP15_GEN-APND.csv")
lmp = df["LMP"]
# lmp for Jan 2nd
lmp_oneday = lmp[17:41]
lmp_oneday

df['INTERVALSTARTTIME_GMT'][17]

'2018-01-02T01:00:00-00:00'

In [26]:
p_dem_df = pd.read_csv("USA_WA_Seattle-Tacoma.727930_TMY2.csv")
p_dem_df.head()
p_dem_df['Date/Time'][47]

' 01/02  24:00:00'

In [27]:
### constants
# time step currently is ONE HOUR
# the day is currently JAN 2nd
# number of hours
n = 24

# power limit
p_lim = 5 * 10 ** 3
# state of charge max
J_max = 20 * 10 ** 3
J_init = 1 * 10 ** 3
# charge efficiency
cha_eff = 0.85
# discharge efficiency
dis_eff = 0.15
# power demand for Jan 2nd
p_dem = p_dem_df["Electricity:Facility [kWh](Hourly)"][23:47]
# power supplied by pv
# calculated for Jan 1, using TMYFormatting.ipynb
p_pv = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 
        0.036874374112418534, 18.874678324745716, 
        15.690924739526677, 72.0408116258169, 
        24.73208104704921, 16.538375698519136, 
        26.69471806669538, 2.0599573028765286, 
        0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]

p_cha = cp.Variable(n)
p_dis = cp.Variable(n)
J = cp.Variable(n)


constraints = [J[0] == J_init + cha_eff * p_cha[0] - p_dis[0] / dis_eff]
constraints += [J[t] == J[t-1] + cha_eff * p_cha[t] - p_dis[t] / dis_eff for t in range(1,n)]
constraints += [J[n-1] >= J_init]
constraints += [p_cha >= 0]
constraints += [p_cha <= p_lim]
constraints += [p_dis >= 0]
constraints += [p_dis <= p_lim]

# convert pandas Series into numpy array
_lmp = lmp_oneday.to_numpy()
p_dem = p_dem.to_numpy()
#J_const = np.array(J_const)

p_net = p_dem - p_pv + p_cha - p_dis

prob = cp.Problem(cp.Minimize(_lmp.T @ p_net), constraints)

prob.solve()
print("A solution p_cha is")
print(p_cha.value)
print("A solution p_dis is")
print(p_dis.value)
print("A solution J is")
print(J.value)


A solution p_cha is
[8.15773886e-15 1.00978896e-14 7.83932506e-15 1.69974968e-14
 2.13169809e-14 2.50350442e-14 3.00108511e-14 2.92373958e-14
 3.13297879e-14 3.30601930e-14 3.51874656e-14 3.14362380e-14
 2.81018128e-14 1.71704557e-14 1.00570019e-14 2.43691173e-14
 2.96400051e-14 3.28668415e-14 3.46223734e-14 3.46048964e-14
 3.12492341e-14 3.02073689e-14 2.38771677e-14 1.70952010e-14]
A solution p_dis is
[2.09716814e-14 1.93151786e-14 2.12692976e-14 1.49679047e-14
 1.30665553e-14 1.17615099e-14 1.03637849e-14 1.05603592e-14
 1.00480970e-14 9.66080691e-15 9.22398150e-15 1.00261356e-14
 1.08657194e-14 1.48899478e-14 1.93557421e-14 1.19856684e-14
 1.04659933e-14 9.71029897e-15 9.34395886e-15 9.34843549e-15
 1.00782248e-14 1.03292225e-14 1.21557314e-14 1.49351150e-14]
A solution J is
[1000. 1000. 1000. 1000. 1000. 1000. 1000. 1000. 1000. 1000. 1000. 1000.
 1000. 1000. 1000. 1000. 1000. 1000. 1000. 1000. 1000. 1000. 1000. 1000.]
